# Exploration of Data of Nextbike-API

### Observations / Assumptions

- we can only see bikes which are free or reserved (booked) right now, we can't see bikes which are currently rented
- available bikes includes the number of bikes which are reserved (booked)
- the number of available bikes is 1349, this is the number we must get after preprocessing
- we assume that there are two different kind of places:
    - bike stations (spot=True and bike=False) where multiple bikes are assigned to one location
    - individual bikes (spot=False and bike=True) where each bike has an own location
- booked_bikes seems to mean the bikes that have been reserved in advance. So bikes_available_to_rent = bikes - booked_bikes
- UIDs of bike stations seem to be in a different range than UIDs of bikes
    
### Open questions

- What is the mapping of the bike types? 71, 150, 183, 196? Maybe one of them code for e-bikes?? Is this relevant for our prediction?
- At a station where there are multiple bikes and some of them are reserved (booked), can we find out which of them is reserved and which is free?
- What are the different place types?

### Preprocessing

- Keep all information from place-, city- and bike-list-section per bike
- Maybe use [JQ](https://stedolan.github.io/jq/manual/) to filter the data and build a JSON that can be easily read by pandas 
- Parse name of place correctly (e.g. Straße)
- Filter out the reserved bikes?

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [4]:
#%matplotlib notebook

In [5]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt 
import json
import plotly.graph_objects as go

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

### Load sample data from API call

In [7]:
file_path = '../sample_data/bikes.json'

In [8]:
with open(file_path, 'r') as f:
    data = json.load(f)

In [9]:
data_leipzig = data['countries'][0]['cities'][0]

In [10]:
data_leipzig

{'uid': 1,
 'lat': 51.3435,
 'lng': 12.3637,
 'zoom': 11,
 'maps_icon': '',
 'alias': 'leipzig',
 'break': False,
 'name': 'Leipzig',
 'num_places': 1317,
 'refresh_rate': '10120',
 'bounds': {'south_west': {'lat': 51.2614, 'lng': 12.241},
  'north_east': {'lat': 51.4479, 'lng': 12.5942}},
 'booked_bikes': 2,
 'set_point_bikes': 1677,
 'available_bikes': 1349,
 'return_to_official_only': True,
 'bike_types': {'71': 598, '150': 213, '183': 13, '196': 539},
 'website': 'https://www.nextbike.de/de/leipzig/',
 'places': [{'uid': 8308,
   'lat': 51.326737,
   'lng': 12.358117,
   'bike': False,
   'name': 'Galopprennbahn (Vorplatz/ZufahrtsstraÃŸe)',
   'address': None,
   'spot': True,
   'number': 4034,
   'booked_bikes': 0,
   'bikes': 0,
   'bikes_available_to_rent': 0,
   'bike_racks': 0,
   'free_racks': 0,
   'special_racks': 0,
   'free_special_racks': 0,
   'maintenance': False,
   'terminal_type': 'free',
   'bike_list': [],
   'bike_numbers': [],
   'bike_types': {},
   'place_typ

In [11]:
places = data_leipzig['places']
print(len(places))

1215


It makes sense that we get less places than available bikes since there are places without any bikes assigned and places with multiple bikes.

In [12]:
bike_stations = [p for p in places if p['spot'] == True and p['bike'] == False]
len(bike_stations)

71

There were 71 stations during this API call, let's see if that number changes when we take an API call with another timestamp.

In [13]:
free_bikes = [p for p in places if p['spot'] == False and p['bike'] == True]
len(free_bikes)

1144

In [30]:
places_with_booked_bikes = [p for p in places if p['bikes_available_to_rent'] < p['bikes']]
places_with_booked_bikes

[{'uid': 92622328,
  'lat': 51.320449,
  'lng': 12.357153,
  'bike': True,
  'name': 'BIKE 21756',
  'address': None,
  'spot': False,
  'number': 0,
  'booked_bikes': 1,
  'bikes': 1,
  'bikes_available_to_rent': 0,
  'bike_racks': 0,
  'free_racks': 0,
  'special_racks': 0,
  'free_special_racks': 0,
  'maintenance': False,
  'terminal_type': '',
  'bike_list': [{'number': '21756',
    'bike_type': 196,
    'lock_types': ['frame_lock'],
    'active': True,
    'state': 'ok',
    'electric_lock': True,
    'boardcomputer': 7551109918,
    'pedelec_battery': None,
    'battery_pack': None}],
  'bike_numbers': ['21756'],
  'bike_types': {'196': 1},
  'place_type': '12',
  'rack_locks': False},
 {'uid': 92702780,
  'lat': 51.340009,
  'lng': 12.374533,
  'bike': True,
  'name': 'BIKE 21776',
  'address': None,
  'spot': False,
  'number': 0,
  'booked_bikes': 1,
  'bikes': 1,
  'bikes_available_to_rent': 0,
  'bike_racks': 0,
  'free_racks': 0,
  'special_racks': 0,
  'free_special_racks

In [15]:
assert len(bike_stations) + len(free_bikes) == len(places), "the number of bike stations and free bikes should add up to the number of all places"
assert len(free_bikes) == len([b for b in free_bikes if b['bike'] == 1]), "following our assumption, there should be no individual bike with a bike count different from 1"

amount_bikes = 0

for p in places:
    amount_bikes += len(p['bike_list'])
    
assert data_leipzig['available_bikes'] == amount_bikes, "following our assumption, the number of available bikes should be equal to the number of bikes appearing in the bike_lists"

amount_booked_bikes = 0

for p in places_with_booked_bikes:
    amount_booked_bikes += p['bikes'] - p['bikes_available_to_rent']
    
assert data_leipzig['booked_bikes'] == amount_booked_bikes, "following our assumption, the number of booked bikes should be equal to the number of total bikes minus the number of bikes available to rent"

In [16]:
bike_stations

[{'uid': 8308,
  'lat': 51.326737,
  'lng': 12.358117,
  'bike': False,
  'name': 'Galopprennbahn (Vorplatz/ZufahrtsstraÃŸe)',
  'address': None,
  'spot': True,
  'number': 4034,
  'booked_bikes': 0,
  'bikes': 0,
  'bikes_available_to_rent': 0,
  'bike_racks': 0,
  'free_racks': 0,
  'special_racks': 0,
  'free_special_racks': 0,
  'maintenance': False,
  'terminal_type': 'free',
  'bike_list': [],
  'bike_numbers': [],
  'bike_types': {},
  'place_type': '17',
  'rack_locks': False},
 {'uid': 16337,
  'lat': 51.343658,
  'lng': 12.378786,
  'bike': False,
  'name': 'Hauptbahnhof/LVB-MobilitÃ¤tszentrale',
  'address': None,
  'spot': True,
  'number': 4024,
  'booked_bikes': 0,
  'bikes': 14,
  'bikes_available_to_rent': 14,
  'bike_racks': 0,
  'free_racks': 0,
  'special_racks': 0,
  'free_special_racks': 0,
  'maintenance': False,
  'terminal_type': 'free',
  'bike_list': [{'number': '20233',
    'bike_type': 71,
    'lock_types': ['frame_lock'],
    'active': True,
    'state': '

### Get Location of bikes

- Extract geo locations from the lists we built out of the JSON file and put them in a dictionary.

In [17]:
free_bike_locations = {}
station_bike_locations = {}

for p in free_bikes:
    for bike in p['bike_list']:
        free_bike_locations.update({bike['number']: {'lat': p['lat'], 'lng': p['lng']}})
        
for p in bike_stations:
    for bike in p['bike_list']:
        station_bike_locations.update({bike['number']: {'lat': p['lat'], 'lng': p['lng']}})

In [18]:
stations = {}
for p in bike_stations:
    stations.update({p['number']: {'name': p['name'], 'lat': p['lat'], 'lng': p['lng'], 'bikes': p['bikes'], 'bikes_available_to_rent': p['bikes_available_to_rent']}})

In [19]:
free_bike_locations_df = pd.DataFrame.from_dict(free_bike_locations, orient='index')
station_bike_locations_df = pd.DataFrame.from_dict(station_bike_locations, orient='index')
stations_df = pd.DataFrame.from_dict(stations, orient='index')

In [20]:
free_bike_locations_df.head()

,lat,lng
21671,51.369716,12.336460
300171,51.334409,12.410872
300379,51.371316,12.383891
300189,51.384498,12.378630
23266,51.375391,12.370363


In [21]:
len(free_bike_locations)

1144

In [22]:
station_bike_locations_df.head()

,lat,lng
20233,51.343658,12.378786
23271,51.343658,12.378786
23201,51.343658,12.378786
39060,51.343658,12.378786
39056,51.343658,12.378786


In [23]:
len(station_bike_locations_df)

205

In [24]:
stations_df.head()

,name,lat,lng,bikes,bikes_available_to_rent
4034,Galopprennbahn (Vorplatz/ZufahrtsstraÃŸe),51.326737,12.358117,0,0
4024,Hauptbahnhof/LVB-MobilitÃ¤tszentrale,51.343658,12.378786,14,14
4042,Hauptbahnhof/Osthalle,51.343813,12.381495,0,0
4046,Hauptbahnhof/Westhalle (LVB MobilitÃ¤tsstation 4),51.344577,12.379628,8,8
4055,Hauptbahnhof/Westseite,51.345639,12.379167,3,3


In [25]:
len(stations_df)

71

- It would be nice to have an additional attribute from type list which contains all related bike numbers.
- Also it would be great to parse the strings correctly so that we can see the "Umlaute".

## Plot bikes on map

In [28]:
mapbox_access_token = "pk.eyJ1IjoiYW5kcmVhc2tpeiIsImEiOiJjbDR1MXBseW4wdnl3M2NudTV2djhmOGRkIn0.ziKnGcQULxdwxhkAjiJBHg"

fig = go.Figure()

# add station markers to map
for index, row in stations_df.iterrows():
    fig.add_trace(go.Scattermapbox(
        lat=[row['lat']],
        lon=[row['lng']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='#1e3799'            
        ),
        text="Station " + str(index),
        name=row['name']
        ))

# add free bike markers to map (only head because it starts lagging otherwise)
for index, row in free_bike_locations_df.head(100).iterrows():
    fig.add_trace(go.Scattermapbox(
        lat=[row['lat']],
        lon=[row['lng']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='#e84118'            
        ),
        text="Bike " + str(index),
        ))
    
fig.update_layout(
    mapbox_style="satellite",
    height=800,
    width=1200,
    margin=go.layout.Margin(
        l=0, # left margin
        r=0, # right margin
        b=0, # bottom margin
        t=0, # top margin
    ),
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=51.34083402293,
            lon=12.375113185144219
        ),
        pitch=0,
        zoom=10
    )
)

fig.update_layout(showlegend=False)

fig.show()

In [27]:
locations = [{'id': b['uid'],'latitude': b['lat'], 'longitude': b['lng'] } for b in bikes]

NameError: name 'bikes' is not defined

In [ ]:
locations_df = pd.DataFrame(locations)

In [ ]:
locations_df.head(5)

### Debug locations

In [ ]:
northest_bike_loc = [northest_bike['latitude'], northest_bike['longitude']]
northest_bike_loc

In [ ]:
northest_bike

In [ ]:
northest_bike = min(locations, key=lambda x: x['longitude'])

#### What's beneath here is trash

In [ ]:
# possible reading with pandas
bike_df = pd.json_normalize(data=data['countries'], record_path=['cities', 'places', 'bike_list'],
                               meta=['lat', 'lng',
                                    'show_bike_types', 'show_bike_type_groups',
                                    'show_free_racks', 'booked_bikes', 'set_point_bikes', 'available_bikes',
                                    ])

# meta information i dropped: pricing, faq_url, name, language, cities, 'store_uri_android', 'store_uri_ios', website, policy, 'system_operator_address', 'country', 'country_name', 'terms', 'policy', 'name', 'hotline'
# domain, language, email, zoom, currency, country_calling_code, no_registration, 'capped_available_bikes'
bike_df.head(3)